### 라이브러리

In [4]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from dotenv import load_dotenv

### API 키

In [5]:
load_dotenv()

# API 키 설정
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")
OPENAI_API_KEY    = os.getenv("OPENAI_API_KEY")

### PDF 로드 / 임베딩

In [6]:
# PDF 로드 및 분할
loader = PyPDFLoader("KB 실버암 간편건강보험Plus.pdf")
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(chunk_size    = 200, 
                                               chunk_overlap = 20,
                                               )
docs = text_splitter.split_documents(pages)

# OpenAI 임베딩
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",
                              openai_api_key = OPENAI_API_KEY,
                              )

# FAISS 벡터 저장소
vectorstore = FAISS.from_documents(docs, 
                                   embedding=embeddings,
                                   )
retriever = vectorstore.as_retriever()

C:\Users\KDT2\AppData\Local\Temp\ipykernel_8856\98390258.py:10: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-3-small",


In [14]:
# Claude LLM
llm = ChatAnthropic(model="claude-opus-4-20250514",
                    temperature=0,
                    max_tokens=1024,
                    api_key=ANTHROPIC_API_KEY,
                    )

# RAG 프롬프트
prompt = hub.pull("rlm/rag-prompt")

# RAG 체인 구성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문
# question = "고지의무를 위반한 경우 보험사는 어떤 조치를 취할 수 있나요?"                                  # 민석 질문
# question = "특정암 ii란 뭐야?"                                                                          # 1번 질문
# question = "유사암진단비iii 를 청구하려고 하는데 갑상선암은 어떤 검사를 통해 진단받아야돼?"                # 2번 질문
# question = "통합암진단비ii 는 보장범위가 어떻게 돼?"                                                      # 3번 질문
# question = "통합전이암진단비의 보장범위는 어떻게 돼?"                                                     # 4번 질문
# question = "보함계약한지 8개월이 지났는데 표적항암방사선 치료비를 보험가입금액의 100%를 받을 수 있어?"      # 5번 질문
# question = "신재진단암진단비ii 에서 수술이란 뭐야?"                                                       # 6번 질문    
question = "보험금청구 했는데 언제 지급되니?"                                                             # 7번 질문

# 1. RAG 기반 응답
rag_answer = rag_chain.invoke(question)

# 2. 순수 LLM 응답 (문서 없이 바로 Claude에게 질문)
llm_only_answer = llm.invoke(question)

# 출력
print("🧾 질문:", question)
print("\n💬 [LLM 단독 응답]:")
print(llm_only_answer)
print("\n📚 [RAG 기반 응답]:")
print(rag_answer)



C:\Users\KDT2\AppData\Roaming\Python\Python311\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


🧾 질문: 보험금청구 했는데 언제 지급되니?

💬 [LLM 단독 응답]:
content='보험금 지급 시기는 보험사와 청구 내용에 따라 다르지만, 일반적인 기준을 안내해드릴게요.\n\n**일반적인 처리 기간:**\n- **단순 청구**: 서류 접수 후 3~7영업일\n- **복잡한 청구**: 10~30일 (조사가 필요한 경우)\n\n**확인하실 사항:**\n1. **서류 완비 여부** - 누락된 서류가 있으면 지연됩니다\n2. **보험사 심사 진행 상황** - 고객센터나 앱에서 확인 가능\n3. **추가 서류 요청** - 문자나 전화 확인\n\n**빠른 처리를 위한 팁:**\n- 보험사 고객센터에 직접 문의\n- 모바일 앱에서 진행상황 확인\n- 담당자 연락처 확보\n\n정확한 일정은 가입하신 보험사에 직접 문의하시는 것이 가장 확실합니다. 보험사명과 청구번호를 준비하시면 더 빠른 확인이 가능해요.' additional_kwargs={} response_metadata={'id': 'msg_01BD6vT5wUHj3QZtYJsKx8CD', 'model': 'claude-opus-4-20250514', 'stop_reason': 'end_turn', 'stop_sequence': None, 'usage': {'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 26, 'output_tokens': 362, 'server_tool_use': None, 'service_tier': 'standard'}, 'model_name': 'claude-opus-4-20250514'} id='run--29d59e59-0bb6-4eee-b52b-91f7d1c98efc-0' usage_metadata={'input_tokens': 26, 'output_tokens': 362, 'total_tokens': 388, 'input_token_details': {'cache_read': 0, 'cach